In [1]:
from pathlib import Path

import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.dataset import Subset
from torchvision.datasets import ImageFolder
from torchvision.transforms import Compose, RandomResizedCrop, RandomVerticalFlip, RandomHorizontalFlip
from torchvision.transforms import ColorJitter, ToTensor, Normalize

FRUIT360_PATH = Path(".") / "fruits-360_dataset" / "fruits-360"

device = "cuda"

train_transform = Compose([
    RandomHorizontalFlip(),    
    RandomResizedCrop(size=32),
    ColorJitter(brightness=0.12),
    ToTensor(),
    Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

val_transform = Compose([
    RandomResizedCrop(size=32),
    ToTensor(),
    Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

batch_size = 128
num_workers = 8

train_dataset = ImageFolder((FRUIT360_PATH /"Training").as_posix(), transform=train_transform, target_transform=None)
val_dataset = ImageFolder((FRUIT360_PATH /"Test").as_posix(), transform=val_transform, target_transform=None)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, 
                          num_workers=num_workers,
                          drop_last=True, pin_memory="cuda" in device)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True,
                        num_workers=num_workers,
                        drop_last=False, pin_memory="cuda" in device)

In [2]:
import mynet
from torchvision.models.squeezenet import squeezenet1_1

#model = squeezenet1_1(pretrained=False, num_classes=93)
model = mynet.ConvNet(num_classes=93)
#model.classifier[-1] = nn.AdaptiveAvgPool2d(1)
model = model.to(device)

In [3]:
from torch.optim import SGD

optimizer = SGD(model.parameters(), lr=0.01, momentum=0.5)
criterion = nn.CrossEntropyLoss()

In [4]:
from ignite.engine import Engine, _prepare_batch

def process_function(engine, batch):
    model.train()
    optimizer.zero_grad()
    x, y = _prepare_batch(batch, device=device)
    y_pred = model(x)
    loss = criterion(y_pred, y)
    loss.backward()
    optimizer.step()
    return loss.item()

trainer = Engine(process_function)

In [5]:
from ignite.engine import Events

log_interval = 50

@trainer.on(Events.ITERATION_COMPLETED)
def log_training_loss(engine):
    iteration = (engine.state.iteration - 1) % len(train_loader) + 1
    if iteration % log_interval == 0:
        print("Epoch[{}] Iteration[{}/{}] Loss: {:.4f}"
              .format(engine.state.epoch, 
                         iteration, 
                         len(train_loader), 
                         engine.state.output))

In [6]:
from ignite.metrics import Loss, CategoricalAccuracy, Precision, Recall

metrics = {
    'avg_loss': Loss(criterion),
    'avg_accuracy': CategoricalAccuracy(),
    'avg_precision': Precision(average=True),
    'avg_recall': Recall(average=True)
}

In [7]:
from ignite.engine import create_supervised_evaluator

train_evaluator = create_supervised_evaluator(model, metrics=metrics, device=device)
val_evaluator = create_supervised_evaluator(model, metrics=metrics, device=device)

In [8]:
indices = np.arange(len(train_dataset))
random_indices = np.random.permutation(indices)[:len(val_dataset)]
train_subset = Subset(train_dataset, indices=random_indices)

train_eval_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True,
                                num_workers=num_workers, 
                                drop_last=True, pin_memory="cuda" in device)

In [9]:
@trainer.on(Events.EPOCH_COMPLETED)
def compute_and_display_offline_train_metrics(engine):
    epoch = engine.state.epoch
    print("Compute train metrics...")
    metrics = train_evaluator.run(train_eval_loader).metrics
    print("Training Results - Epoch: {}  Average Loss: {:.4f} | Accuracy: {:.4f} | Precision: {:.4f} | Recall: {:.4f}"
          .format(engine.state.epoch, 
                      metrics['avg_loss'], 
                      metrics['avg_accuracy'], 
                      metrics['avg_precision'], 
                      metrics['avg_recall']))

@trainer.on(Events.EPOCH_COMPLETED)
def compute_and_display_val_metrics(engine):
    epoch = engine.state.epoch
    print("Compute validation metrics...")
    metrics = val_evaluator.run(val_loader).metrics
    print("Validation Results - Epoch: {}  Average Loss: {:.4f} | Accuracy: {:.4f} | Precision: {:.4f} | Recall: {:.4f}"
          .format(engine.state.epoch, 
                      metrics['avg_loss'], 
                      metrics['avg_accuracy'], 
                      metrics['avg_precision'], 
                      metrics['avg_recall']))

In [10]:
from torch.optim.lr_scheduler import ExponentialLR

lr_scheduler = ExponentialLR(optimizer, gamma=1)

@trainer.on(Events.EPOCH_STARTED)
def update_lr_scheduler(engine):
    lr_scheduler.step()
    # Вывод значений скорости обучения:
    if len(optimizer.param_groups) == 1:
        lr = float(optimizer.param_groups[0]['lr'])
        print("Learning rate: {}".format(lr))
    else:
        for i, param_group in enumerate(optimizer.param_groups):
            lr = float(param_group['lr'])
            print("Learning rate (group {}): {}".format(i, lr))

In [12]:
from ignite.handlers import ModelCheckpoint

def score_function(engine):
    val_avg_accuracy = engine.state.metrics['avg_accuracy']
    return val_avg_accuracy

best_model_saver = ModelCheckpoint("best_models",  
                                   filename_prefix="model",
                                   score_name="val_accuracy",  
                                   score_function=score_function,
                                   n_saved=3,
                                   save_as_state_dict=True,
                                   create_dir=True)


val_evaluator.add_event_handler(Events.COMPLETED, 
                                best_model_saver, 
                                {"best_model": model})

training_saver = ModelCheckpoint("checkpoint",
                             filename_prefix="checkpoint",
                             save_interval=1000,
                             n_saved=1,
                             save_as_state_dict=True,
                             create_dir=True)

to_save = {"model": model, "optimizer": optimizer, "lr_scheduler": lr_scheduler} 
trainer.add_event_handler(Events.ITERATION_COMPLETED, training_saver, to_save)

In [13]:
from ignite.handlers import EarlyStopping

early_stopping = EarlyStopping(patience=10, 
                              score_function=score_function, 
                              trainer=trainer)

val_evaluator.add_event_handler(Events.EPOCH_COMPLETED, early_stopping)

In [14]:
max_epochs = 100
output = trainer.run(train_loader, max_epochs=max_epochs)

Learning rate: 0.01
Epoch[1] Iteration[50/371] Loss: 4.3245
Epoch[1] Iteration[100/371] Loss: 3.6551
Epoch[1] Iteration[150/371] Loss: 3.0498
Epoch[1] Iteration[200/371] Loss: 2.7232
Epoch[1] Iteration[250/371] Loss: 2.5947
Epoch[1] Iteration[300/371] Loss: 1.9119
Epoch[1] Iteration[350/371] Loss: 1.7364
Compute train metrics...
Training Results - Epoch: 1  Average Loss: 1.8238 | Accuracy: 0.4764 | Precision: 0.5693 | Recall: 0.4680
Compute validation metrics...
Validation Results - Epoch: 1  Average Loss: 1.8302 | Accuracy: 0.4675 | Precision: 0.5509 | Recall: 0.4593
Learning rate: 0.01
Epoch[2] Iteration[50/371] Loss: 1.7889
Epoch[2] Iteration[100/371] Loss: 1.6482
Epoch[2] Iteration[150/371] Loss: 1.3564
Epoch[2] Iteration[200/371] Loss: 1.5517
Epoch[2] Iteration[250/371] Loss: 1.3361
Epoch[2] Iteration[300/371] Loss: 1.4403
Epoch[2] Iteration[350/371] Loss: 1.0390
Compute train metrics...
Training Results - Epoch: 2  Average Loss: 1.2786 | Accuracy: 0.6343 | Precision: 0.6799 | Rec

Epoch[15] Iteration[150/371] Loss: 0.2121
Epoch[15] Iteration[200/371] Loss: 0.3122
Epoch[15] Iteration[250/371] Loss: 0.2413
Epoch[15] Iteration[300/371] Loss: 0.3980
Epoch[15] Iteration[350/371] Loss: 0.1487
Compute train metrics...
Training Results - Epoch: 15  Average Loss: 0.2196 | Accuracy: 0.9328 | Precision: 0.9353 | Recall: 0.9313
Compute validation metrics...
Validation Results - Epoch: 15  Average Loss: 0.2464 | Accuracy: 0.9237 | Precision: 0.9270 | Recall: 0.9215
Learning rate: 0.01
Epoch[16] Iteration[50/371] Loss: 0.2198
Epoch[16] Iteration[100/371] Loss: 0.3493
Epoch[16] Iteration[150/371] Loss: 0.2241
Epoch[16] Iteration[200/371] Loss: 0.2095
Epoch[16] Iteration[250/371] Loss: 0.1343
Epoch[16] Iteration[300/371] Loss: 0.2219
Epoch[16] Iteration[350/371] Loss: 0.3085
Compute train metrics...
Training Results - Epoch: 16  Average Loss: 0.2254 | Accuracy: 0.9290 | Precision: 0.9325 | Recall: 0.9280
Compute validation metrics...
Validation Results - Epoch: 16  Average Loss

Epoch[29] Iteration[200/371] Loss: 0.1488
Epoch[29] Iteration[250/371] Loss: 0.1602
Epoch[29] Iteration[300/371] Loss: 0.2135
Epoch[29] Iteration[350/371] Loss: 0.2044
Compute train metrics...
Training Results - Epoch: 29  Average Loss: 0.1468 | Accuracy: 0.9534 | Precision: 0.9552 | Recall: 0.9531
Compute validation metrics...
Validation Results - Epoch: 29  Average Loss: 0.1549 | Accuracy: 0.9510 | Precision: 0.9521 | Recall: 0.9496
Learning rate: 0.01
Epoch[30] Iteration[50/371] Loss: 0.2169
Epoch[30] Iteration[100/371] Loss: 0.1903
Epoch[30] Iteration[150/371] Loss: 0.1675
Epoch[30] Iteration[200/371] Loss: 0.2515
Epoch[30] Iteration[250/371] Loss: 0.1496
Epoch[30] Iteration[300/371] Loss: 0.1450
Epoch[30] Iteration[350/371] Loss: 0.2284
Compute train metrics...
Training Results - Epoch: 30  Average Loss: 0.1404 | Accuracy: 0.9543 | Precision: 0.9559 | Recall: 0.9550
Compute validation metrics...
Validation Results - Epoch: 30  Average Loss: 0.1688 | Accuracy: 0.9476 | Precision: 0

In [15]:
class TestDataset(Dataset):

    def __init__(self, ds):
        self.ds = ds

    def __len__(self):
        return len(self.ds)

    def __getitem__(self, index):
        return self.ds[index][0], index

test_dataset = TestDataset(val_dataset)

test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False,
                         num_workers=num_workers, 
                         drop_last=False, pin_memory="cuda" in device)

In [16]:
import torch.nn.functional as F
from ignite._utils import convert_tensor

def _prepare_batch(batch):
    x, index = batch
    x = convert_tensor(x, device=device)
    return x, index

def inference_update(engine, batch):
    x, indices = _prepare_batch(batch)
    y_pred = model(x)
    y_pred = F.softmax(y_pred, dim=1)
    return {"y_pred": convert_tensor(y_pred, device='cpu'), "indices": indices}

model.eval()
inferencer = Engine(inference_update) 

In [26]:
@inferencer.on(Events.EPOCH_COMPLETED)
def log_tta(engine):
    print("TTA {} / {}".format(engine.state.epoch, n_tta))

n_tta = 3
num_classes = 93
n_samples = len(val_dataset)

# Массив для хранения предсказаний
y_probas_tta = np.zeros((n_samples, num_classes, n_tta), dtype=np.float32)

@inferencer.on(Events.ITERATION_COMPLETED)
def save_results(engine):
    output = engine.state.output
    tta_index = engine.state.epoch - 1
    start_index = ((engine.state.iteration - 1) % len(test_loader)) * batch_size
    end_index = min(start_index + batch_size, n_samples)
    batch_y_probas = output['y_pred'].detach().numpy()
    y_probas_tta[start_index:end_index, :, tta_index] = batch_y_probas

In [27]:
model = mynet.ConvNet(num_classes=93)
#model.classifier[-1] = nn.AdaptiveAvgPool2d(1)
model = model.to(device)

model_state_dict = torch.load("best_models/model_best_model_32_val_accuracy=0.9559937.pth")
model.load_state_dict(model_state_dict)

In [ ]:
inferencer.run(test_loader, max_epochs=n_tta)

In [24]:
y_probas = np.mean(y_probas_tta, axis=-1)
y_preds = np.argmax(y_probas, axis=-1)

In [ ]:
from sklearn.metrics import accuracy_score

y_test_true = [y for _, y in val_dataset]
accuracy_score(y_test_true, y_preds)